In [42]:
from operator import add, mul
from typing   import Union
from helpers import data
import ast
import re

In [2]:
expressions = data(18, parser=lambda s: s.replace(" ", ""))
expressions[:3]

['5+3+(7*9+8+8)+9', '4+2*((2+8*7*9)*7*(2+4)+(7+9+5+6*2*6))', '5+8*5*(6+8*7)*5']

**Part 1:** Evaluate the expression in each line. The operators have no precedence, except for parentheses. Calculate the sum of all the expressions. 

This is probably best done recursively. Process a line left to right, until I hit a parentheses and then recurse to evaluate the expression inside it.

In [34]:
def evaluate(expression: Iterator[chr]):
    """Evaluate expression, which should yield each atom (operation, 
    parentheses, or one-digit number) one at a time.
    """
    if iter(expression) is not expression:
        raise TypeError("expression must be an Iter")
    result = 0 
    operation = add # First operation is to add the first number to 0
    while atom := next(expression, None):
        if atom == ")":
            break 

        if atom == "+":
            operation = add 
        elif atom == "*":
            operation = mul 
        elif atom == "(":
            result = operation(result, evaluate(expression))
        else: 
            result = operation(result, int(atom))
            
    return result

assert evaluate(iter("2*3+(4*5)")) == 26
assert evaluate(iter("((2+4*9)*(6+9*8+6)+6)+2+4*2")) == 13632

In [35]:
iterator_expressions = [iter(expression) for expression in expressions]

In [36]:
sum(evaluate(iterator_expression) for iterator_expression in iterator_expressions)

8929569623593

**Part 1, Norvig:** Avoid using an iterator. Notice we need a way to recurse and evaluate a parenthetical section, but then have the parent continue executing *after* the parenthetical section. Okay, I couldn't figure out a nice way to do this, so I looked at Norvig's solution, which is perfect. 

In [56]:
# Ugh, again I'm reminded that getting the data in the proper 
# format is half the battle. Norvig has it in tuples which 
# treates parenthetical expressions as one expression. This 
# way his solution also deals with numbers that are more than 
# one digit. 

def parse_expression(line: str) -> tuple:
    """Parse each expression into a tuple where each element 
    is a valid atom or expression. 
    "1 + 2 * (3 * 4)" -> (1, "+", (3, "*", 4))
    """
    # Convert all +, * to '+', '*'
    # \1 copies the string we matched
    line_string_operators = re.sub("([+*])", r", '\1', ", line)
    # Convert into a tuple by evaluating the string
    return ast.literal_eval(line_string_operators)

assert parse_expression("123 + 9") == (123, "+", 9)
assert parse_expression("123 + 9 * (1 * 3 + 4)") == (123, "+", 9, "*", (1, "*", 3, "+", 4))

ops = {"+": add, "*": mul}

expressions = data(18, parser=parse_expression)
expressions[0]

(5, '+', 3, '+', (7, '*', 9, '+', 8, '+', 8), '+', 9)

In [59]:
def evaluate(expression: Union[int, tuple]) -> int: 
    """Evaluate the expression one operation at a time, giving all 
    operators the same precedence. 
    """
    if isinstance(expression, int):
        return expression 
    else: 
        a, op, b, *rest = expression
        result = ops[op]( evaluate(a), evaluate(b) )
        return evaluate((result, *rest)) if rest else result

In [60]:
sum(evaluate(expression) for expression in expressions)

8929569623593

**Part 2:** Now addition has higher precedence than multiplication. Find the sum of all the expressions. 

I can't even think of a way to do this with my iterator solution because to do it in one pass, I need to peek ahead to see if the next operation would have higher preference. It's easy enough to do with Norvig's parsed data though. 

In [63]:
def evaluate(expression: Union[int, tuple]) -> int:
    """Evaluate the expression one operation at a time, with + 
    having higher precedence than *. We can satisfy this 
    requirement by peeking at the next operation: 
        a op1 b op2 *rest 
    if op2 is higher precedence, evaluate b op2 *rest first. 
    """
    if isinstance(expression, int):
        return expression
    else: 
        a, op, b, *rest = expression 
        if rest: 
            # Get next operator and operand
            next_op, c, *rest = rest
            if next_op == "+" and op == "*":
                # Do next operation first 
                result = ops[next_op]( evaluate(b), evaluate(c) )
                return evaluate((a, op, result, *rest))
            else: 
                # Continue left-to-right 
                result = ops[op]( evaluate(a), evaluate(b) )
                return evaluate((result, next_op, c, *rest))
        else: 
            return ops[op]( evaluate(a), evaluate(b) )
        
assert evaluate((2, "*", 3, "+", (4, "*", 5))) == 46

In [64]:
sum(evaluate(expression) for expression in expressions)

231235959382961

**Part 2, Norvig:** This solution is much shorter than mine, but I'm not sure I like it better. Going left-to-right is intuitive for me. 

In [67]:
def evaluate(expression: Union[int, tuple]) -> int:
    if isinstance(expression, int):
        return expression
    elif "*" in expression: 
        # Evaluate this after everything else 
        pos = expression.index("*")
        return evaluate(expression[:pos]) * evaluate(expression[pos + 1:])
    else: 
        # It's just sums left 
        return sum(evaluate(atom) for atom in expression if atom != "+")
        
assert evaluate((2, "*", 3, "+", (4, "*", 5))) == 46

In [68]:
sum(evaluate(expression) for expression in expressions)

231235959382961